# Course Module

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import csv

import json
import unittest

from common import testExit, projdir

from entrant import Entrant
from period import Period
from speedrun import SpeedRun

from constants import *

## Course Class

Class to manage courses - start / end times

In [2]:
class Course(Period):
    def __init__(self, session, courseId, startTime, endTime, verbosity=1):
        '''Initialise course object'''

        super().__init__(parent=session, verbosity=verbosity)
        
        self.date = session.date

        self.courseId = courseId
        self.courseName = '{} {}'.format(self.appConfig['Courses'][courseId[:1]], courseId[1:])

        self.startTime = startTime
        self.endTime = endTime

        self.minStartTime = '23:59:59'
        self.maxStartTime = '00:00:00'


    def storeValidRun(self, entrantId, speedRun, startTime):
        '''Store run data from CSV files'''

        if startTime:
            if startTime < self.minStartTime:
                self.minStartTime = startTime
            if startTime > self.maxStartTime:
                self.maxStartTime = startTime

            if startTime >= self.startTime and startTime <= self.endTime:
                self.storeRun(entrantId, speedRun)
        else:
            self.storeRun(entrantId, speedRun)


    def loadRunData(self, csvPath):
        '''Read run data from CSV files'''

        prevSailNo = None

        with open(csvPath, 'r') as f:
            csvReader = csv.reader(f)
            headers = colNames = next(csvReader)
            headersPlus = [T_COURSE] + headers

            if T_SAIL_NUMBER not in headers:
                raise ValueError('Field "{}" missing in "{}"'.format(T_SAIL_NUMBER, os.path.basename(csvPath)))

            sailNoIndex = headers.index(T_SAIL_NUMBER)

            # Some early years do not have the start times of runs so use whatever is available!
            if 'Start Time' in headers:
                startTimeIndex = headers.index('Start Time')
            elif 'Time' in headers:
                startTimeIndex = headers.index('Time')
            elif 'Finish Time' in headers:
                startTimeIndex = headers.index('Finish Time')
            else:
                startTimeIndex = -1

            # Name may be used for secondary lookup
            if 'Name' in headers:
                nameIndex = headers.index('Name')
            else:
                nameIndex = -1

            if 'FirstName' in headers:
                firstNameIndex = headers.index('FirstName')
            elif 'First Name' in headers:
                firstNameIndex = headers.index('First Name')
            else:
                firstNameIndex = -1

            if 'LastName' in headers:
                lastNameIndex = headers.index('LastName')
            elif 'Last Name' in headers:
                lastNameIndex = headers.index('Last Name')
            else:
                lastNameIndex = -1

            for values in csvReader:
                if len(values) != len(headers):
                    raise ValueError('Incorrect number of fields in "{}" - {}'.format(os.path.basename(csvPath), values))

                # Sail number always upper case
                sailNo = values[sailNoIndex]
                if startTimeIndex >= 0:
                    startTime = values[startTimeIndex]
                else:
                    startTime = None

                # Determine name for secondary match
                if nameIndex >= 0:
                    name = values[nameIndex].strip()                  
                elif firstNameIndex >= 0 and lastNameIndex >= 0:
                    firstName = values[firstNameIndex].strip()
                    lastName = values[lastNameIndex].strip()
                    if firstName and lastName:
                        name = firstName + ' ' + lastName
                    elif firstName:
                        name = firstName
                    elif lastName:
                        name = lastName
                    else:
                        name = None
                else:
                    name = None

                # Quick hack for 2009
                if startTime == 'GPS':
                    startTime = '12:00:00'

                # Lookup is required less often if data is sorted by sail number
                if sailNo != prevSailNo:
                    entrant = self.getEntrantBySailNo(sailNo, name)
                    entrantId = entrant.getValue('ID')
                    prevSailNo = sailNo

                # Ensure course name is stored in the result itself for the benefit of reporting
                valuesPlus = [self.courseName] + values

                # Run details are stored in a dedicated object
                speedRun = SpeedRun(self, entrant, headersPlus, valuesPlus, verbosity=self.verbosity)

                # Store the run if the start time is valid
                self.storeValidRun(entrantId, speedRun, startTime)


    def loadGpsData(self, csvPath):
        '''Read GPS data from CSV files'''

        prevGt31Id = None
        headers = [T_RUN, T_FILENAME, T_START_TIME, T_DURATION, T_SPEED, T_COG]
        headersPlus = [T_COURSE] + headers

        with open(csvPath, 'r') as f:
            csvReader = csv.reader(f)
            for values in csvReader:
                if len(values) != len(headers):
                    raise ValueError('Incorrect number of fields in "{}" - {}'.format(os.path.basename(csvPath), values))

                # GPSResults does not output a header line
                run, filename, startTime, duration, speed, cog = values

                # Split up record and format the GT-31 details
                try:
                    gt31Id, gt31Serial, fileDate, fileTime = os.path.splitext(filename)[0].split('_')[:4]
                except:
                    self.logError('Problem parsing GT-31 details in run data - "{}"'.format(filename))
                    raise
                gt31Id = gt31Id.upper()

                # Lookup is required less often if data is sorted by GT-31 ID
                if gt31Id != prevGt31Id:
                    entrant = self.getEntrantByGt31(gt31Id, gt31Serial)
                    entrantId = entrant.getValue('ID')
                    prevGt31Id = gt31Id

                # Ensure course name is stored in the result itself for the benefit of reporting
                valuesPlus = [self.courseName] + values
                
                # Run details are stored in a dedicated object
                speedRun = SpeedRun(self, entrant, headersPlus, valuesPlus, verbosity=self.verbosity)

                # Store the run if the start time is valid
                self.storeValidRun(entrantId, speedRun, startTime)


    def loadResults(self, csvPath):
        '''Read results data from CSV files'''
        
        # Temporary hack which works for years prior to 2010
        self.loadRunData(csvPath)
        

    def finaliseRuns(self):
        '''Final processing after loading the runs'''

        self.logInfo('{} runs by {} participants down course {} on {} - {} to {}'.format(
            self.numRuns, len(self.runs), self.courseId, self.date, self.minStartTime, self.maxStartTime))

        if self.minStartTime < self.startTime:
            self.logWarning('Runs found before course {} opened on {} - earliest was {}'.format(
                self.courseId, self.date, self.minStartTime))

        if self.maxStartTime > self.endTime:
            self.logWarning('Runs found after course {} closed on {} - latest was {}'.format(
                self.courseId, self.date, self.maxStartTime))

        self.sortRuns()


    def loadRuns(self, csvPath):
        '''Read speeds from CSV files'''

        csvName = os.path.basename(csvPath)
        prefix = csvName.split('_')[0].upper()

        if prefix == 'RUNDATA':
            self.loadRunData(csvPath)
        elif prefix == 'GPSDATA':
            self.loadGpsData(csvPath)
        elif prefix == 'RESULTS':
            self.loadResults(csvPath)
        else:
            raise ValueError('Unexpected prefix "{}" for {}'.format(prefix, csvName))

        self.finaliseRuns()

## Unit Tests

A handful of very basic tests, including a dummy session class

In [3]:
class DummySession(Period):
    def __init__(self, sessionDate):
        '''Initialise session object'''
        
        super().__init__()

        self.date = sessionDate
        
        self.entrants[0] = Entrant()

        self.appConfig = appConfig

In [4]:
class TestRunData20001003(unittest.TestCase):
    '''Class to test Course class'''
    
    def testRunData20001003_S1(self, session=None):
        '''Test RUNDATA using 20001003 S1 data'''

        if session is None:
            sessionDate = '20001003'
            session = DummySession(sessionDate)
        else:
            sessionDate = session.date

        courseId = 'S1'
        csvPath = os.path.join(projdir, EVENTS_DIR, sessionDate[:4], RUNDATA_DIR, sessionDate,
                        'RUNDATA_{}_{}.csv'.format(sessionDate, courseId))

        # Vebosity is zero to suppress 'WARNING: Unrecognised sail number' 
        course = Course(session, courseId, '09:00:00', '18:00:00', verbosity=0)
        course.loadRuns(csvPath)
        
        self.assertEqual(course.numRuns, 246)
        self.assertEqual(len(course.runs), 26)

        # Check runs are sorted correctly
        for personId in course.runs:
            maxSpeed = 99.999
            for run in course.runs[personId]:
                self.assertEqual(run.data[T_SPEED] <= maxSpeed, True)
                maxSpeed = run.data[T_SPEED]


    def testRunData20001003_H1(self, session=None):
        '''Test RUNDATA using 20001003 H1 data'''

        if session is None:
            sessionDate = '20001003'
            session = DummySession(sessionDate)
        else:
            sessionDate = session.date

        courseId = 'H1'
        csvPath = os.path.join(projdir, EVENTS_DIR, sessionDate[:4], RUNDATA_DIR, sessionDate,
                        'RUNDATA_{}_{}.csv'.format(sessionDate, courseId))

        # Vebosity is zero to suppress 'WARNING: Unrecognised sail number' 
        course = Course(session, courseId, '09:00:00', '18:00:00', verbosity=0)
        course.loadRuns(csvPath)
        
        self.assertEqual(course.numRuns, 124)
        self.assertEqual(len(course.runs), 14)

        # Check runs are sorted correctly
        for personId in course.runs:
            maxSpeed = 99.999
            for run in course.runs[personId]:
                self.assertEqual(run.data[T_SPEED] <= maxSpeed, True)
                maxSpeed = run.data[T_SPEED]


    def testRunData20001003(self):
        '''Test RUNDATA using test20001003 data'''

        sessionDate = '20001003'
        session = DummySession(sessionDate)

        self.testRunData20001003_S1(session=session)
        self.testRunData20001003_H1(session=session)

        self.assertEqual(session.numRuns, 370)
        self.assertEqual(len(session.runs), 30)

In [5]:
class TestResults20001003(unittest.TestCase):
    '''Class to test Course class'''
    
    def testResults20001003_S1(self, session=None):
        '''Test RESULTS using 20001003 S1 data'''

        if session is None:
            sessionDate = '20001003'
            session = DummySession(sessionDate)
        else:
            sessionDate = session.date

        courseId = 'S1'
        csvPath = os.path.join(projdir, EVENTS_DIR, sessionDate[:4], 'results', sessionDate,
                        'RESULTS_{}_{}.csv'.format(sessionDate, courseId))

        # Vebosity is zero to suppress 'WARNING: Unrecognised sail number' 
        course = Course(session, courseId, '09:00:00', '18:00:00', verbosity=0)
        course.loadRuns(csvPath)
        
        self.assertEqual(course.numRuns, len(course.runs))


    def testResults20001003_H1(self, session=None):
        '''Test RESULTS using 20001003 H1 data'''

        if session is None:
            sessionDate = '20001003'
            session = DummySession(sessionDate)
        else:
            sessionDate = session.date

        courseId = 'S1'
        csvPath = os.path.join(projdir, EVENTS_DIR, sessionDate[:4], 'results', sessionDate,
                        'RESULTS_{}_{}.csv'.format(sessionDate, courseId))

        # Vebosity is zero to suppress 'WARNING: Unrecognised sail number' 
        course = Course(session, courseId, '09:00:00', '18:00:00', verbosity=0)
        course.loadRuns(csvPath)
        
        self.assertEqual(course.numRuns, len(course.runs))

In [6]:
class TestGpsData20191011(unittest.TestCase):
    '''Class to test Course class'''
    
    def testGpsData20191011_S1(self, session=None):
        '''Test GPSDATA using 20191011 S1 data'''

        if session is None:
            sessionDate = '20191011'
            session = DummySession(sessionDate)
        else:
            sessionDate = session.date

        courseId = 'S1'
        csvPath = os.path.join(projdir, EVENTS_DIR, sessionDate[:4], GPSDATA_DIR, sessionDate,
                        'GPSDATA_{}_{}.csv'.format(sessionDate, courseId))

        # Vebosity is zero to suppress 'WARNING: Unrecognised GT-31 ID' 
        course = Course(session, courseId, '09:00:00', '17:00:00', verbosity=0)
        course.loadRuns(csvPath)
        
        self.assertEqual(course.numRuns, 3)
        self.assertEqual(len(course.runs), 3)

        # Check runs are sorted correctly
        for personId in course.runs:
            maxSpeed = 99.999
            for run in course.runs[personId]:
                self.assertEqual(run.data[T_SPEED] <= maxSpeed, True)
                maxSpeed = run.data[T_SPEED]


    def testGpsData20191011_S2(self, session=None):
        '''Test GPSDATA using 20191011 S2 data'''

        if session is None:
            sessionDate = '20191011'
            session = DummySession(sessionDate)
        else:
            sessionDate = session.date

        courseId = 'S2'
        csvPath = os.path.join(projdir, EVENTS_DIR, sessionDate[:4], GPSDATA_DIR, sessionDate,
                        'GPSDATA_{}_{}.csv'.format(sessionDate, courseId))

        # Vebosity is zero to suppress 'WARNING: Unrecognised GT-31 ID'
        course = Course(session, courseId, '09:00:00', '17:00:00', verbosity=0)
        course.loadRuns(csvPath)
        
        self.assertEqual(course.numRuns, 512)
        self.assertEqual(len(course.runs), 36)

        # Check runs are sorted correctly
        for personId in course.runs:
            maxSpeed = 99.999
            for run in course.runs[personId]:
                self.assertEqual(run.data[T_SPEED] <= maxSpeed, True)
                maxSpeed = run.data[T_SPEED]


    def testGpsData20191011(self):
        '''Test GPSDATA using 20191011 data'''

        sessionDate = '20191011'
        session = DummySession(sessionDate)

        self.testGpsData20191011_S1(session=session)
        self.testGpsData20191011_S2(session=session)

        self.assertEqual(session.numRuns, 515)
        self.assertEqual(len(session.runs), 36)

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [7]:
if __name__ == '__main__':
    # Read main config into global variable
    filename = os.path.join(projdir, CONFIG_DIR, CONFIG_JSON)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        appConfig = json.loads(jsonTxt)

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

........
----------------------------------------------------------------------
Ran 8 tests in 0.107s

OK


## All Done!